In [1]:
# Dependencies and Setup
import pandas as pd
import requests
import gmaps

# Import API key
from config import g_key

# Configure gmaps API key
gmaps.configure(api_key=g_key)

In [2]:
# 1. Import the WeatherPy_database.csv file. 
city_data_df = pd.read_csv("../Weather_Database/WeatherPy_database.csv")
city_data_df.head()

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
0,0,Murdochville,CA,48.9583,-65.5006,24.30,77,92,24.00,overcast clouds
1,1,Qaanaaq,GL,77.4840,-69.3632,-4.02,68,56,3.33,broken clouds
2,2,Hermanus,ZA,-34.4187,19.2345,67.89,78,100,10.71,overcast clouds
3,3,Mataura,NZ,-46.1927,168.8643,48.47,94,82,1.03,broken clouds
4,4,Bethel,US,41.3712,-73.4140,47.43,38,1,4.00,clear sky


In [3]:
# 2. Prompt the user to enter minimum and maximum temperature criteria 
min_temp = float(input("What is the minimum temperature you would like for your trip? "))
max_temp = float(input("What is the maximum temperature you would like for your trip? "))

What is the minimum temperature you would like for your trip? 75
What is the maximum temperature you would like for your trip? 90


In [5]:
# 3. Filter the city_data_df DataFrame using the input statements to create a new DataFrame using the loc method.
preferred_cities_df = city_data_df.loc[(city_data_df["Max Temp"] <= max_temp) 
                                    & (city_data_df["Max Temp"] >= min_temp)]
preferred_cities_df.head(30)

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
6,6,Mahebourg,MU,-20.4081,57.7000,79.09,69,40,11.50,scattered clouds
8,8,Castro,BR,-24.7911,-50.0119,77.88,44,94,5.41,overcast clouds
10,10,Hobyo,SO,5.3505,48.5268,79.05,80,61,19.39,broken clouds
12,12,Rikitea,PF,-23.1203,-134.9692,77.49,75,72,9.28,broken clouds
13,13,Riohacha,CO,11.5444,-72.9072,86.63,55,98,14.00,overcast clouds
23,23,Samarai,PG,-10.6167,150.6667,81.77,81,100,12.62,overcast clouds
24,24,Sao Filipe,CV,14.8961,-24.4956,75.65,69,20,9.95,few clouds
25,25,Bandarbeyla,SO,9.4942,50.8122,77.11,80,6,24.07,clear sky
27,27,Georgetown,MY,5.4112,100.3354,81.90,80,40,8.05,scattered clouds
28,28,Sao Joao Da Barra,BR,-21.6403,-41.0511,86.09,80,77,18.41,broken clouds


In [6]:
# 4a. Determine if there are any empty rows.
preferred_cities_df.count()

City_ID                203
City                   203
Country                203
Lat                    203
Lng                    203
Max Temp               203
Humidity               203
Cloudiness             203
Wind Speed             203
Current Description    203
dtype: int64

In [7]:
# 4b. Drop any empty rows and create a new DataFrame that doesn’t have empty rows.
preferred_cities_clean_df = preferred_cities_df.dropna()
preferred_cities_clean_df.count()

City_ID                203
City                   203
Country                203
Lat                    203
Lng                    203
Max Temp               203
Humidity               203
Cloudiness             203
Wind Speed             203
Current Description    203
dtype: int64